# Battle of the Neighborhoods
## Comparison of the Communities in Trinidad and Tobago

Install the required libraries for the analysis:

In [1]:
import requests
import pandas as pd
import numpy as np

import json # library to handle JSON files
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium
import folium

!pip install geopandas
!pip install geopy
from geopy.geocoders import Nominatim

print('Libraries Imported.')

  Using cached geopandas-0.8.1-py2.py3-none-any.whl (962 kB)
  Using cached pyproj-3.0.0.post1-cp38-cp38-win_amd64.whl (14.4 MB)

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\User\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-hppi0gub\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-hppi0gub\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\User\AppData\Local\Temp\pip-pip-egg-info-392pgd70'
         cwd: C:\Users\User\AppData\Local\Temp\pip-install-hppi0gub\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.



  Using cached Fiona-1.8.17.tar.gz (1.3 MB)
Libraries Imported.


### 1. Gathering the Data
---
The data was scraped from the site https://www.citypopulation.de/en/trinidad/admin/ which has up to date data on the population of cities around the world. These numbers were compared with the census data from the Central Statistical Office of Trinidad and Tobago (https://cso.gov.tt/subjects/population-and-vital-statistics/population/). This was placed in a dataframe for easier handling.

In [11]:
url = 'https://www.citypopulation.de/en/trinidad/admin/'
read_table = pd.read_html(url, header=[0])
df = read_table[0]
df.head()

,Name,Status,PopulationCensus2000-05-15,PopulationCensus2011-01-09,Unnamed: 4
0,Arima,Borough,32278,33606,→
1,Arima Proper,Urban Community,9129,9082,→
2,Calvary Hill,Urban Community,1165,1330,→
3,Carib Homes,Urban Community,465,585,→
4,Malabar,Urban Community,8218,11130,→


The new dataframe was cleaned and formatted.

In [12]:
df.drop(['PopulationCensus2000-05-15', 'Unnamed: 4'], axis=1, inplace=True)
df.rename(columns={'PopulationCensus2011-01-09':'Population'}, inplace=True)
df

,Name,Status,Population
0,Arima,Borough,33606
1,Arima Proper,Urban Community,9082
2,Calvary Hill,Urban Community,1330
3,Carib Homes,Urban Community,585
4,Malabar,Urban Community,11130
...,...,...,...
605,Valley View,Urban Community,1056
606,Valsayn,Urban Community,2552
607,Wallerfield,Rural Community,4218
608,Warren Village,Urban Community,2125


In [13]:
df.drop([609], inplace=True)
df

,Name,Status,Population
0,Arima,Borough,33606
1,Arima Proper,Urban Community,9082
2,Calvary Hill,Urban Community,1330
3,Carib Homes,Urban Community,585
4,Malabar,Urban Community,11130
...,...,...,...
604,Tunapuna,Urban Community,16701
605,Valley View,Urban Community,1056
606,Valsayn,Urban Community,2552
607,Wallerfield,Rural Community,4218


### 2. Collecting Location Data
---

Other methods to determine the latitudes and longitudes did not work with this dataset and such the followind code utilizing `geocoder` had to be used. Firstly, an Address column was created, which consisted of the name of the community and Trinidad and Tobago.

In [14]:
df['Address'] = df['Name'].astype(str) + ', Trinidad and Tobago'
df.head()

,Name,Status,Population,Address
0,Arima,Borough,33606,"Arima, Trinidad and Tobago"
1,Arima Proper,Urban Community,9082,"Arima Proper, Trinidad and Tobago"
2,Calvary Hill,Urban Community,1330,"Calvary Hill, Trinidad and Tobago"
3,Carib Homes,Urban Community,585,"Carib Homes, Trinidad and Tobago"
4,Malabar,Urban Community,11130,"Malabar, Trinidad and Tobago"


The following code used `geocoder` from `geopy` to get the location data using the Address column and create new columns within the dataframe to store the newly collected data.

In [16]:
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent='myGeocoder')

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

df['location'] = df['Address'].apply(geocode)

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('White Land, Trinidad and Tobago',), **{}).
Traceback (most recent call last):
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/http/client.py", line 1369, in getresponse
    response.begin()
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/http/client.py", line 310, in begin
    version, status, reason = self._read_status()
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/http/client.py", line 271, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/opt/conda/envs/Python-3.7-main/lib/python3.7/socket.py", line 589, in rea

In [17]:
print(df.shape)
df.head()

(609, 9)


,Name,Status,Population,Address,location,point,latitude,longitude,altitude
0,Arima,Borough,33606,"Arima, Trinidad and Tobago","(Arima, 300831, Trinidad and Tobago, (10.63720...","(10.6372057, -61.2829762, 0.0)",10.637206,-61.282976,0.0
1,Arima Proper,Urban Community,9082,"Arima Proper, Trinidad and Tobago",None,None,NaN,NaN,NaN
2,Calvary Hill,Urban Community,1330,"Calvary Hill, Trinidad and Tobago","(Calvary Hill, Saint Ann's, Port of Spain, 190...","(10.7097744, -61.5090987, 0.0)",10.709774,-61.509099,0.0
3,Carib Homes,Urban Community,585,"Carib Homes, Trinidad and Tobago",None,None,NaN,NaN,NaN
4,Malabar,Urban Community,11130,"Malabar, Trinidad and Tobago","(Malabar, Arima, 300831, Trinidad and Tobago, ...","(10.6196194, -61.2774636, 0.0)",10.619619,-61.277464,0.0


The unnecessary columns were removed and those communities without any location data were also removed.

In [45]:
df.dropna(inplace=True)
df.drop(['Unnamed: 0', 'Address', 'location', 'point', 'altitude'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Name,Status,Population,latitude,longitude
0,Arima,Borough,33606,10.637206,-61.282976
1,Calvary Hill,Urban Community,1330,10.709774,-61.509099
2,Malabar,Urban Community,11130,10.619619,-61.277464
3,Mount Pleasant,Urban Community,1240,11.167020,-60.792943
4,O'Meara Road,Urban Community,2593,10.592523,-61.294142


In [46]:
df.shape

(479, 5)

### 3. Display All Points on Map
---

A map was created using `folium` to display all the communities to be used in the analysis.

In [47]:
tt_address = 'Trinidad and Tobago'
locator = Nominatim(user_agent='myGeocoder')

location = locator.geocode(tt_address)
tt_latitude = location.latitude
tt_longitude = location.longitude
print(str(tt_latitude) + ',' + str(tt_longitude))

10.8677845,-60.9821067


In [48]:
tt_map = folium.Map(location=[tt_latitude, tt_longitude], zoom_start=9)

for lat, lng, status, population, name in zip(df['latitude'], df['longitude'], df['Status'], df['Population'], df['Name']):
    label = '{}: {}, {}'.format(name, status, population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    fill=True,
    fill_color='blue',
    fill_opacity=0.6,
    parse_html=False).add_to(tt_map)
    
tt_map

### 4. Initializing Foursquare API to Gather Data
---

The credentials were entered to get the nearby venues for all the communities.

In [43]:
# The code was removed by Watson Studio for sharing.

#### Explore the Neighborhoods in Toronto
Get the latitude and longitude values of the venues in the communities. The limit for the number of venues was set at 100 and the radius as 500 metres. A GET request URL was then created. A GET request was sent and the results were examined to determine where the information required is. A function was created (from the Week 3 tutorial) to repeat the process for all the communities in Trinidad and Tobago.

In [23]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

A new dataframe, _tt_venues_ was created.

In [24]:
tt_venues = getNearbyVenues(names=df['Name'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Arima
Calvary Hill
Malabar
Mount Pleasant
O'Meara Road
Tumpuna Road
Chaguanas
Charlieville
Edinburgh 500
Edinburgh Gardens
Endeavour Village
Enterprise
Esmeralda
Felicity
Homeland Gardens
Jerningham Junction
Lange Park
Lendore Village
Montrose Village
Munroe Settlement
Petersfield
Couva/Tabaquite/Talparo
Agostini Village
Arena
Balmain
Basta Hall
Bonne Aventure
Brasso Caparo Village
Brasso Tamana
Brasso Venado
Brechin Castle
Brickfield
Brothers Road
Butler Village
Calcutta Road No. 2
Calcutta Settlement No. 2
California
Caparo
Carapichaima
Caratal
Carlsen Field
Cedar Hill
Chandernagore
Chase Village
Chickland
Chin Chin
Claxton Bay
Coalmine
Corosal
Couva Central
Diamond
Dow Village
Edinburgh Village
Esperanza
Fairview
Farnum Village
Felicity Hall
Flanagin Town
Forres Park
Freeport
Friendship
Gasparillo
Gran Couva
Guaracara
Hermitage
Indian Trail
Macaulay
Madras Settlement
Mamoral No. 2
Mayo
Mc Bean
Mount Pleasant
Mundo Nuevo
Nancoo Village
Orange Valley
Palmiste
Parforce
Pepper Village
P

In [25]:
print(tt_venues.shape)
tt_venues.head()

(1560, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arima,10.637206,-61.282976,Pennywise cosmetics,10.639299,-61.280745,Cosmetics Shop
1,Arima,10.637206,-61.282976,Pizza Hut,10.639227,-61.281888,Pizza Place
2,Arima,10.637206,-61.282976,Bo's Chinese Restaurant,10.635889,-61.281402,Chinese Restaurant
3,Arima,10.637206,-61.282976,Pizza Hut - Windsor,10.639398,-61.282298,Pizza Place
4,Arima,10.637206,-61.282976,Hi Lo Ridgewood,10.638998,-61.282862,Supermarket


The number of venues for each community returned:

In [26]:
tt_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alyce Glen,6,6,6,6,6,6
Aranguez,5,5,5,5,5,5
Arena,1,1,1,1,1,1
Arima,9,9,9,9,9,9
Aripero Village,1,1,1,1,1,1
...,...,...,...,...,...,...
Waddle Village,4,4,4,4,4,4
Wellington,3,3,3,3,3,3
West Moorings,2,2,2,2,2,2


In [27]:
print('There are {} uniques categories.'.format(len(tt_venues['Venue Category'].unique())))

There are 181 uniques categories.


### 5. Analyzing Each Community
---
Dummy variables for each category were created.

In [28]:
# one hot encoding
tt_onehot = pd.get_dummies(tt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tt_onehot['Neighborhood'] = tt_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tt_onehot.columns[-1]] + list(tt_onehot.columns[:-1])
tt_onehot = tt_onehot[fixed_columns]

tt_onehot.head()

,Yoga Studio,Airport,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Travel & Transport,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Water Park,Waterfall,Waterfront,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
tt_onehot.shape

(1560, 181)

Group rows by Neighborhood and take the mean of the frequency of occurrence of each category.

In [30]:
tt_grouped = tt_onehot.groupby('Neighborhood').mean().reset_index()
tt_grouped

,Neighborhood,Yoga Studio,Airport,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Travel & Transport,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Water Park,Waterfall,Waterfront,Wine Shop,Wings Joint,Women's Store
0,Alyce Glen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,Aranguez,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.2,0.0
2,Arena,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Arima,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,Aripero Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,Waddle Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
295,Wellington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0
296,West Moorings,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
297,Whim,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [31]:
tt_grouped.shape

(299, 181)

Determine the top 5 most common venues in each neighborhood.

In [32]:
num_top_venues = 5

for hood in tt_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tt_grouped[tt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alyce Glen----
                 venue  freq
0    Indian Restaurant  0.17
1   Chinese Restaurant  0.17
2  Fried Chicken Joint  0.17
3          Supermarket  0.17
4             Pharmacy  0.17


----Aranguez----
                           venue  freq
0             Chinese Restaurant   0.4
1              Indian Restaurant   0.2
2                    Wings Joint   0.2
3                 Breakfast Spot   0.2
4  Paper / Office Supplies Store   0.0


----Arena----
                           venue  freq
0                  Grocery Store   1.0
1                    Yoga Studio   0.0
2  Paper / Office Supplies Store   0.0
3                      Multiplex   0.0
4                    Music Store   0.0


----Arima----
                venue  freq
0         Pizza Place  0.22
1                Café  0.11
2      Cosmetics Shop  0.11
3  Chinese Restaurant  0.11
4         Supermarket  0.11


----Aripero Village----
           venue  freq
0     Sports Bar   1.0
1  Movie Theater   0.0
2  Moving Target   0.0
3 

                  venue  freq
0              Pharmacy  0.33
1                   Bar  0.33
2         Grocery Store  0.33
3           Yoga Studio  0.00
4  Other Great Outdoors  0.00


----Carapichaima----
                  venue  freq
0                   Bar   1.0
1           Yoga Studio   0.0
2  Other Great Outdoors   0.0
3             Multiplex   0.0
4           Music Store   0.0


----Carenage----
                  venue  freq
0         Grocery Store  0.50
1            Steakhouse  0.25
2  Fast Food Restaurant  0.25
3           Yoga Studio  0.00
4          Optical Shop  0.00


----Carlsen Field----
         venue  freq
0  Flower Shop   1.0
1  Yoga Studio   0.0
2    Multiplex   0.0
3  Music Store   0.0
4  Music Venue   0.0


----Carrera Island----
                           venue  freq
0                  Grocery Store   1.0
1                    Yoga Studio   0.0
2  Paper / Office Supplies Store   0.0
3                      Multiplex   0.0
4                    Music Store   0.0


----Cas

           venue  freq
0     Steakhouse  0.25
1            Bar  0.25
2  Grocery Store  0.25
3         Market  0.25
4    Yoga Studio  0.00


----Freeport----
                           venue  freq
0                            Bar  0.75
1                     Restaurant  0.25
2                    Yoga Studio  0.00
3  Paper / Office Supplies Store  0.00
4                    Music Store  0.00


----Fyzabad----
               venue  freq
0  Indian Restaurant  0.33
1               Bank  0.33
2             Bakery  0.33
3      Movie Theater  0.00
4          Multiplex  0.00


----Gasparillo----
                  venue  freq
0  Fast Food Restaurant   0.4
1         Shopping Mall   0.2
2           Pizza Place   0.2
3    Chinese Restaurant   0.2
4           Yoga Studio   0.0


----George Village----
           venue  freq
0       Mountain   1.0
1  Movie Theater   0.0
2      Multiplex   0.0
3    Music Store   0.0
4    Music Venue   0.0


----Glencoe----
                venue  freq
0          Food Cou

                  venue  freq
0            Hotel Pool  0.33
1  Caribbean Restaurant  0.33
2         Grocery Store  0.33
3                 Plaza  0.00
4            Playground  0.00


----Lucy Vale----
                  venue  freq
0  Fast Food Restaurant  0.25
1         Grocery Store  0.12
2           Pizza Place  0.12
3    Chinese Restaurant  0.12
4           Salad Place  0.12


----Macoya----
                           venue  freq
0             Chinese Restaurant  0.36
1              Convenience Store  0.18
2  Paper / Office Supplies Store  0.09
3                           Park  0.09
4  Vegetarian / Vegan Restaurant  0.09


----Mafeking----
                           venue  freq
0                     Campground   1.0
1                    Yoga Studio   0.0
2  Paper / Office Supplies Store   0.0
3                    Music Store   0.0
4                    Music Venue   0.0


----Mahoe----
                  venue  freq
0     Convenience Store  0.33
1           Music Venue  0.33
2         

                  venue  freq
0     Indian Restaurant  0.25
1  Fast Food Restaurant  0.25
2              Pharmacy  0.25
3        Sandwich Place  0.25
4  Other Great Outdoors  0.00


----Petersfield----
           venue  freq
0       Boutique   1.0
1    Yoga Studio   0.0
2  Moving Target   0.0
3    Music Store   0.0
4    Music Venue   0.0


----Petit Bourg----
                           venue  freq
0                            Bar   0.4
1              Convenience Store   0.2
2             Chinese Restaurant   0.2
3                    Snack Place   0.2
4  Paper / Office Supplies Store   0.0


----Petit Café----
                           venue  freq
0             Chinese Restaurant   1.0
1                    Yoga Studio   0.0
2  Paper / Office Supplies Store   0.0
3                    Music Store   0.0
4                    Music Venue   0.0


----Petit Morne----
               venue  freq
0         Food Truck   0.2
1        Auto Garage   0.2
2                Bar   0.2
3  Convenience Stor

                venue  freq
0  Chinese Restaurant  0.18
1            Pharmacy  0.18
2              Bakery  0.09
3        Burger Joint  0.09
4         Bus Station  0.09


----Talparo----
                  venue  freq
0                Bakery   1.0
1           Yoga Studio   0.0
2  Other Great Outdoors   0.0
3             Multiplex   0.0
4           Music Store   0.0


----Tamana Road----
                  venue  freq
0                Bakery   1.0
1           Yoga Studio   0.0
2  Other Great Outdoors   0.0
3             Multiplex   0.0
4           Music Store   0.0


----Tarouba----
                  venue  freq
0    Seafood Restaurant   1.0
1           Yoga Studio   0.0
2  Other Great Outdoors   0.0
3             Multiplex   0.0
4           Music Store   0.0


----Techier Village----
           venue  freq
0            Spa   1.0
1    Yoga Studio   0.0
2  Movie Theater   0.0
3      Multiplex   0.0
4    Music Store   0.0


----Todd's Road----
                  venue  freq
0                 

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
tt_venues_sorted = pd.DataFrame(columns=columns)
tt_venues_sorted['Neighborhood'] = tt_grouped['Neighborhood']

for ind in np.arange(tt_grouped.shape[0]):
    tt_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tt_grouped.iloc[ind, :], num_top_venues)

tt_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alyce Glen,Sports Bar,Chinese Restaurant,Pharmacy,Indian Restaurant,Supermarket,Fried Chicken Joint,Eastern European Restaurant,Field,Fast Food Restaurant,Farmers Market
1,Aranguez,Chinese Restaurant,Wings Joint,Indian Restaurant,Breakfast Spot,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
2,Arena,Grocery Store,Women's Store,Diner,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
3,Arima,Pizza Place,Cosmetics Shop,Chinese Restaurant,Fast Food Restaurant,Basketball Court,Supermarket,Martial Arts School,Café,Electronics Store,Field
4,Aripero Village,Sports Bar,Women's Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service,Electronics Store


### 6. Cluster Data
---
Use k-means to cluster the neighborhood into 5 clusters

In [37]:
# set number of clusters
kclusters = 5

tt_grouped_clustering = tt_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 0])

In [40]:
# add clustering labels
tt_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tt_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tt_merged = tt_merged.join(tt_venues_sorted.set_index('Neighborhood'), on='Name')

tt_merged.head() # check the last columns!

,Name,Status,Population,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arima,Borough,33606,10.637206,-61.282976,1.0,Pizza Place,Cosmetics Shop,Chinese Restaurant,Fast Food Restaurant,Basketball Court,Supermarket,Martial Arts School,Café,Electronics Store,Field
1,Calvary Hill,Urban Community,1330,10.709774,-61.509099,1.0,Fast Food Restaurant,Gas Station,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Farmers Market,Farm,Falafel Restaurant,Event Service
2,Malabar,Urban Community,11130,10.619619,-61.277464,0.0,Bar,Deli / Bodega,BBQ Joint,Women's Store,Event Service,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
3,Mount Pleasant,Urban Community,1240,11.167020,-60.792943,1.0,Auto Garage,Stadium,Diner,Women's Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
4,O'Meara Road,Urban Community,2593,10.592523,-61.294142,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


Visuzlize the clusters on a map of Trinidad and Tobago.

In [22]:
kclusters = 5
tt_merged.dropna(inplace=True)
# create map
map_clusters = folium.Map(location=[tt_latitude, tt_longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tt_merged['latitude'], tt_merged['longitude'], tt_merged['Name'], tt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 7. Examine Clusters
---
#### Cluster 1

In [42]:
tt_merged.loc[tt_merged['Cluster Labels'] == 0, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Status,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Urban Community,0.0,Bar,Deli / Bodega,BBQ Joint,Women's Store,Event Service,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
6,Borough,0.0,Bar,Furniture / Home Store,Dessert Shop,Asian Restaurant,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
7,Urban Community,0.0,Café,Business Service,Cosmetics Shop,Bar,BBQ Joint,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
11,Urban Community,0.0,Convenience Store,Chinese Restaurant,Bar,Fried Chicken Joint,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
12,Urban Community,0.0,Coffee Shop,Bar,Lounge,Liquor Store,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
13,Urban Community,0.0,Bar,Nightclub,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
14,Urban Community,0.0,Coffee Shop,Bar,BBQ Joint,Liquor Store,Flea Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
16,Urban Community,0.0,Indian Restaurant,Gym / Fitness Center,Caribbean Restaurant,Bar,Park,Women's Store,Eastern European Restaurant,Field,Fast Food Restaurant,Farmers Market
17,Urban Community,0.0,Convenience Store,Chinese Restaurant,Bar,Fried Chicken Joint,Women's Store,Electronics Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
32,Semi-Urban Community,0.0,Electronics Store,Caribbean Restaurant,Bar,Spa,Women's Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm


#### Cluster 2

In [43]:
tt_merged.loc[tt_merged['Cluster Labels'] == 1, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Status,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Borough,1.0,Pizza Place,Cosmetics Shop,Chinese Restaurant,Fast Food Restaurant,Basketball Court,Supermarket,Martial Arts School,Café,Electronics Store,Field
1,Urban Community,1.0,Fast Food Restaurant,Gas Station,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Farmers Market,Farm,Falafel Restaurant,Event Service
3,Urban Community,1.0,Auto Garage,Stadium,Diner,Women's Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
8,Urban Community,1.0,Convenience Store,Arcade,Burger Joint,Gym / Fitness Center,Bar,Furniture / Home Store,Event Service,Fish & Chips Shop,Field,Fast Food Restaurant
9,Urban Community,1.0,Chinese Restaurant,Fast Food Restaurant,Bakery,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Farmers Market,Farm,Falafel Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
469,Urban Community,1.0,Spa,Event Service,Convenience Store,Middle Eastern Restaurant,Bar,Women's Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
471,Urban Community,1.0,Pharmacy,Chinese Restaurant,Bus Station,Fast Food Restaurant,Park,Basketball Court,Burger Joint,Bakery,Electronics Store,Women's Store
472,Urban Community,1.0,Pizza Place,Mobile Phone Shop,Pet Store,Electronics Store,Korean Restaurant,Playground,Cosmetics Shop,Convenience Store,Fast Food Restaurant,Farmers Market
474,Urban Community,1.0,Fast Food Restaurant,Cosmetics Shop,Indian Restaurant,Farmers Market,Park,Sandwich Place,Burger Joint,Electronics Store,Women's Store,Eastern European Restaurant


#### Cluster 3

In [44]:
tt_merged.loc[tt_merged['Cluster Labels'] == 2, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Status,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
183,Urban Community,2.0,Chinese Restaurant,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
223,Urban Community,2.0,Chinese Restaurant,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
225,Urban Community,2.0,Chinese Restaurant,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
461,Urban Community,2.0,Chinese Restaurant,Women's Store,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service


#### Cluster 4

In [45]:
tt_merged.loc[tt_merged['Cluster Labels'] == 3, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Status,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
38,Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
53,Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
59,Urban Community,3.0,Bar,Restaurant,Women's Store,Diner,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
68,Rural Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
75,Semi-Urban Community,3.0,Bar,Bakery,Women's Store,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm
76,Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
91,Semi-Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
92,Semi-Urban Community,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
136,Region,3.0,Bar,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


#### Cluster 5

In [46]:
tt_merged.loc[tt_merged['Cluster Labels'] == 4, tt_merged.columns[[1] + list(range(5, tt_merged.shape[1]))]]

,Status,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,Urban Community,4.0,Beach,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
269,Rural Community,4.0,Beach,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
295,Rural Community,4.0,Beach,Scenic Lookout,Women's Store,Diner,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
318,Semi-Urban Community,4.0,Pool,Beach,Eastern European Restaurant,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Service
334,Rural Community,4.0,Beach,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
358,Urban Community,4.0,Beach,Women's Store,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


### 8. Analyzing and Comparing Data
---
Using the population data from the above dataframe, crime data obtained from the Central Statistics Ofiice of Trinidad and Tobago for the year 2018 (https://cso.gov.tt/wp-content/uploads/2020/01/Total-Number-of-Serious-Crimes-by-Police-Division-Station-2018.xls.xls), a new dataframe was created. This focused on the broader regions of Trinidad and Tobago. The crime data deals with serious crimes reported to Police Stations in the respective districts.

In [29]:
file = 'table1.csv'
df1 = pd.read_csv(file)
df1

,Name,Population,Area,Population Density,Serious Crimes Reported
0,Arima,33606,11.69,2874.76,685
1,Chaguanas,83516,60.07,1390.31,945
2,Couva/Tabaquite/Talparo,178410,748.40,238.39,1116
3,Diego Martin,102957,126.81,811.90,1023
4,Mayaro/Rio Claro,35650,784.90,45.42,199
5,Penal/Debe,89392,245.97,363.43,195
6,Point Fortin,20235,27.38,739.04,172
7,Port of Spain,37074,13.13,2823.61,1277
8,Princes Town,102375,617.71,165.73,623
9,San Fernando,48838,20.14,2424.93,1133


A .geojson file was created distinguishing the boundaries of each district.

In [30]:
trinidad_geo = r'new.geojson'

Align the names within the .geojson file to the dataframe so that it can be incorporated seamlessly.

In [31]:
with open(trinidad_geo) as trinidad_file:
    trinidad_json = json.load(trinidad_file)
    
names_json = []
for index in range(len(trinidad_json['features'])):
    names_json.append(trinidad_json['features'][index]['properties']['Name'])
    
names_json

['Arima',
 'Chaguanas',
 'Couva/Tabaquite/Talparo',
 'Mayaro/Rio Claro',
 'Princes Town',
 'San Fernando',
 'Penal/Debe',
 'Point Fortin',
 'Siparia',
 'Port of Spain',
 'San Juan/Laventille',
 'Tunapuna/Piarco',
 'Sangre Grande',
 'Deigo Martin']

In [32]:
dataframe_names = df1.Name.tolist()

# names in the json file - the same order as in the data frame 
geojson_names = ['Arima', 'Chaguanas', 'Couva/Tabaquite/Talparo', 'Deigo Martin', 'Mayaro/Rio Claro', 'Penal/Debe', 'Point Fortin', 'Port of Spain', 'Princes Town', 'San Fernando', 'Sangre Grande', 'San Juan/Laventille', 'Siparia', 'Tunapuna/Piarco']

# replace data frame names by json names
df1.replace(dict(zip(dataframe_names, geojson_names)), inplace=True)
df1

,Name,Population,Area,Population Density,Serious Crimes Reported
0,Arima,33606,11.69,2874.76,685
1,Chaguanas,83516,60.07,1390.31,945
2,Couva/Tabaquite/Talparo,178410,748.40,238.39,1116
3,Deigo Martin,102957,126.81,811.90,1023
4,Mayaro/Rio Claro,35650,784.90,45.42,199
5,Penal/Debe,89392,245.97,363.43,195
6,Point Fortin,20235,27.38,739.04,172
7,Port of Spain,37074,13.13,2823.61,1277
8,Princes Town,102375,617.71,165.73,623
9,San Fernando,48838,20.14,2424.93,1133


#### Map of Trinidad showing Population Data by District
A choropleth map was created to show the population of each district in Trinidad.

In [37]:
threshold_scale = np.linspace(df1['Population'].min(),
                              df1['Population'].max(),
                              6, dtype=int)
threshold_scale = threshold_scale.tolist() # change the numpy array to a list
threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# let Folium determine the scale.
pop_map = folium.Map(location=[10.536421, -61.311951], height= 1500, width=1300, zoom_start=10)
pop_map.choropleth(
    geo_data=trinidad_geo,
    data=df1,
    columns=['Name', 'Population'],
    key_on='feature.properties.Name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Trinidad Population by Municipality',
    reset=True
)
pop_map

The clusters were overlayed on the population choropleth map to determine if any patterns could be distinguished.

In [51]:
markers_colors = []
for lat, lon, poi, cluster in zip(tt_merged['latitude'], tt_merged['longitude'], tt_merged['Name'], tt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(pop_map)

pop_map

#### Map of Trinidad showing 2018 Crime Data by District
A choropleth map was created to show the number of reported serious crimes for each district in Trinidad.

In [55]:
threshold1_scale = np.linspace(df1['Serious Crimes Reported'].min(),
                              df1['Serious Crimes Reported'].max(),
                              6, dtype=int)
threshold1_scale = threshold1_scale.tolist() # change the numpy array to a list
threshold1_scale[-1] = threshold1_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# let Folium determine the scale.
crime_map = folium.Map(location=[10.536421, -61.311951], zoom_start=10)
crime_map.choropleth(
    geo_data=trinidad_geo,
    data=df1,
    columns=['Name', 'Serious Crimes Reported'],
    key_on='feature.properties.Name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Trinidad Crime by Municipality',
    reset=True
)
crime_map

The clusters were overlayed on the crime choropleth map to determine if any patterns could be distinguished.

In [24]:
markers_colors = []
for lat, lon, poi, cluster in zip(tt_merged['latitude'], tt_merged['longitude'], tt_merged['Name'], tt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(crime_map)

crime_map